In [ ]:
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bunzip2 "shape_predictor_68_face_landmarks.dat.bz2"

In [ ]:
import numpy as np
import cv2
import dlib
import os
import matplotlib.pyplot as plt
%matplotlib inline
import math
import imutils
from imutils import face_utils

Predictor_path = '/content/shape_predictor_68_face_landmarks.dat'
IMAGE_PATH = ''   ####### PASS IMAGE PATH
BASENAME = os.path.basename(IMAGE_PATH).split(".")[0]
FOLDER_PATH = os.path.join(os.path.dirname(IMAGE_PATH),BASENAME) ####### PATH TO SAVE IMAGES

if not os.path.exists(FOLDER_PATH):
  os.mkdir(FOLDER_PATH)


detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(Predictor_path)

###Getting Landmarks####

class TooManyFaces(Exception):
  pass
class NoFaces(Exception):
  pass

def get_landmarks1(im):
  rects = detector(im, 1)

  if len(rects)>1:
    raise TooManyFaces
  if len(rects) == 0:
    raise NoFaces
  
  for i in rects:
    #cv2.rectangle(im, (i.left(),i.top()), (i.right(), i.bottom()), 255, 1)
    shape = predictor(im, i)
    for j in range(shape.num_parts):
      p = shape.part(j)
      #cv2.circle(im, (p.x,p.y), 2,255,1)
      #cv2.putText(im, str(i),(p.x +4,p.y), cv2.FONT_HERSHEY_SIMPLEX, .25, (255,255,255))
  
  im1 = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
  plt.imshow(im1)

  return np.matrix([[p.x,p.y] for p in predictor(im, rects[0]).parts()]), rects[0], shape

#''' LEFT CHEEK'''
def Left_Cheek(im):
  left_cheek = [*range(6),48,60,49,31]
  left_cheek_cood = np.matrix([mat[x] for x in left_cheek])
  remap_left_cheek = remapped_img(left_cheek_cood)
  #print(remap_left_cheek)
  feat_mask = np.zeros((im.shape[0], im.shape[1]))

  cv2.fillConvexPoly(feat_mask, remap_left_cheek, 1)
  feat_mask = feat_mask.astype(np.bool)
  out_face = np.zeros((im.shape[0], im.shape[1]))
  out_face[feat_mask] = im[feat_mask]

  return remove_black_background(out_face)


#''' RIGHT CHEEK''''
def Right_Cheek(im):
  right_cheek = [*range(15,10,-1),54,64,53,35]
  right_cheek_cood = np.matrix([mat[x] for x in right_cheek])
  remap_left_cheek = remapped_img(right_cheek_cood)
  #print(remap_left_cheek)
  feat_mask = np.zeros((im.shape[0], im.shape[1]))

  cv2.fillConvexPoly(feat_mask, remap_left_cheek, 1)
  feat_mask = feat_mask.astype(np.bool)
  out_face = np.zeros((im.shape[0], im.shape[1]))
  out_face[feat_mask] = im[feat_mask]

  return remove_black_background(out_face)

#''' CHIN ''''
def Chin(im):
  chin = [*range(59,54,-1),*range(10,5,-1)]
  chin_cood = np.matrix([mat[x] for x in chin])
  remap_left_cheek = remapped_img(chin_cood)
  #print(remap_left_cheek)
  feat_mask = np.zeros((im.shape[0], im.shape[1]))

  cv2.fillConvexPoly(feat_mask, remap_left_cheek, 1)
  feat_mask = feat_mask.astype(np.bool)
  out_face = np.zeros((im.shape[0], im.shape[1]))
  out_face[feat_mask] = im[feat_mask]

  return remove_black_background(out_face)

#'''' FOREHEAD''''
def Forehead(im,shape1,mat):

  feat_mask1 = 255* np.ones((im.shape[0], im.shape[1]), dtype =np.uint8)
  #feat_mask1.fill(255)
  cv2.imwrite('/content/white.jpg',feat_mask1)
  white_im = cv2.imread('/content/white.jpg')

  remap_im = remapped_img(shape1)
  cv2.fillConvexPoly(feat_mask1, remap_im[0:27], 0)

  [vx,vy,x,y] = cv2.fitLine(np.matrix([mat[0],mat[16]]),cv2.DIST_L2,0,0.01,0.01)

  test_im = test_img.copy()
  test_im[int(x):,:] = 0
  test_im[:,mat[16][0]:] = 0
  test_im[:,:mat[0][0]]

  feat_mask1 = feat_mask1.astype(np.bool)
  #print(feat_mask1[0])
  out_face1 = np.zeros((test_im.shape[0], test_im.shape[1]))
  out_face1[feat_mask1] = test_im[feat_mask1]
      
  #plt.imshow(out_face1, cmap='gray')
  

  return remove_black_background(out_face1)

#####DRIVER CODE#####

test_img = cv2.imread(IMAGE_PATH,0)
mat, d ,shape1= get_landmarks1(test_img)

shape1 = face_utils.shape_to_np(shape1)
mat = mat.tolist()
#print(mat)

remap_img = remapped_img(shape1)
feat_mask = np.zeros((test_img.shape[0], test_img.shape[1]))

cv2.fillConvexPoly(feat_mask, remap_img[0:27], 1)
feat_mask = feat_mask.astype(np.bool)
out_face = np.zeros((test_img.shape[0], test_img.shape[1]))
out_face[feat_mask] = test_img[feat_mask]
out_face = remove_black_background(out_face)
plt.subplot(3,3,1)
plt.imshow(out_face, cmap='gray')

'''left_cheek = [*range(6),48,60,49,31]
right_cheek = [*range(15,10,-1),54,64,53,35]
chin = [*range(59,54,-1),*range(10,5,-1)]
print(left_cheek)
print(right_cheek)
print(chin)

left_cheek_cood = np.matrix([mat[x] for x in left_cheek])
right_cheek_cood = np.matrix([mat[x] for x in right_cheek])
chin_cood = np.matrix([mat[x] for x in chin])
'''
LEFT_CHEEK = os.path.join(FOLDER_PATH,BASENAME+'_LEFTCHEEK.jpg')
RIGHT_CHEEK = os.path.join(FOLDER_PATH,BASENAME+'_RIGHTCHEEK.jpg')
CHIN = os.path.join(FOLDER_PATH,BASENAME+'_CHIN.jpg')
FOREHEAD = os.path.join(FOLDER_PATH,BASENAME+'_FOREHEAD.jpg')

remap_left_cheek = Left_Cheek(test_img)
cv2.imwrite(LEFT_CHEEK,remap_left_cheek)
plt.subplot(3,3,2)
plt.imshow(remap_left_cheek, cmap='gray')

remap_right_cheek = Right_Cheek(test_img)
cv2.imwrite(RIGHT_CHEEK,remap_right_cheek)
plt.subplot(3,3,3)
plt.imshow(remap_right_cheek, cmap = 'gray')

remap_chin = Chin(test_img)
cv2.imwrite(CHIN,remap_chin)
plt.subplot(3,3,4)
plt.imshow(remap_chin, cmap='gray')

remap_foreheaad = Forehead(test_img,shape1,mat)
cv2.imwrite(FOREHEAD,remap_foreheaad)
plt.subplot(3,3,5)
plt.imshow(remap_foreheaad, cmap ="gray")
plt.show()
